In [33]:
from elasticsearch import Elasticsearch, helpers
import os
import re
import json
import logging
import operator

In [34]:
def connect_elasticsearch():
    _es = None
    _es = Elasticsearch([{'host': 'localhost', 'port': 9200}])
    if _es.ping():
        print('Yay Connected')
    else:
        print('Awww it could not connect!')
    return _es
if __name__ == '__main__':
  logging.basicConfig(level=logging.ERROR)

In [35]:
es = connect_elasticsearch()

Yay Connected


## Load Query File 

In [36]:
query_file_path = "D:\\CS 6200\\AP_DATA\\query_desc.51-100.short.txt"
queries = dict()
with open(query_file_path) as f:
    for line in f:
        if len(line.strip()) > 0:
            query_no = re.search(r'\d+', line).group()
            line = line.lstrip('0123456789.- ')
            queries[query_no] = line.rstrip()

## Load Results file for TF-IDF

In [37]:
tfidf_file_path = "D:\\CS 6200\\AP_DATA\\results_tfidf.txt"
doc_dict = dict()
relevant_docs = []
line_num = 0
query_num = 0
with open(tfidf_file_path) as f:
    for line in f:
        line_num +=1
        tokens = line.split()
        query_num = tokens[0]
        if line_num == 1000:
            line_num = 0
            doc_dict[query_num] = relevant_docs
            relevant_docs = []
        if line_num in range(1,11):
            relevant_docs.append(tokens[2])
        else:
            continue
        

In [38]:
print(doc_dict.keys())

dict_keys(['85', '59', '56', '71', '64', '62', '93', '99', '58', '77', '54', '87', '94', '100', '89', '61', '95', '68', '57', '97', '98', '60', '80', '63', '91'])


In [39]:
query_relevant_terms = dict()

In [40]:
for key in doc_dict:
    statistics = es.mtermvectors(index = "assignment1", ids = doc_dict[key], term_statistics = False, fields = "text")
    relevant_terms = dict()
    for doc in statistics["docs"]:
        if doc["term_vectors"]:
                words = doc["term_vectors"]["text"]["terms"]
                termsList = list(words.keys())
                for term in termsList:
                    if term not in queries[key].split():
                        if term not in relevant_terms:
                            relevant_terms[term] = words[term]["term_freq"]
                        else:
                            relevant_terms[term] += words[term]["term_freq"]
    sorted_d = dict( sorted(relevant_terms.items(), key=operator.itemgetter(1),reverse=True))
    sortedKeys = list(sorted_d)

    query_relevant_terms[key] = sortedKeys[:3]   
print(query_relevant_terms)       


{'85': ['bribe', 'govern', 'charg'], '59': ['wind', 'hurrican', 'damag'], '56': ['percent', 'bank', 'interest'], '71': ['south', 'u.n', 'namibia'], '64': ['hostag', 'report', 'releas'], '62': ['govern', 'militari', 'avril'], '93': ['rifl', 'gun', 'weapon'], '99': ['north', 'iran', 'contra'], '58': ['worker', 'railwai', 'transport'], '77': ['eleph', 'ivori', 'trade'], '54': ['satellit', 'rocket', 'orbit'], '87': ['financi', 'investig', 'hous'], '94': ['comput', 'sai', 'inform'], '100': ['soviet', 'compani', 'gorbachev'], '89': ['oil', 'opec', 'price'], '61': ['north', 'iran', 'arm'], '95': ['comput', 'ibm', 'compani'], '68': ['compani', 'worker', 'safeti'], '57': ['amp', 'mci', 't'], '97': ['cabl', 'compani', 'telephon'], '98': ['cabl', 'compani', 'telephon'], '60': ['pai', 'rais', 'feder'], '80': ['presidenti', 'candid', 'fund'], '63': ['translat', 'comput', 'machin'], '91': ['weapon', 'chemic', 'armi']}


In [41]:
print(queries['85'])

allegation corrupt official government bribes


In [42]:
outF = open("D:\\CS 6200\\AP_DATA\\queries_pr1.txt", "a")
for key in queries:
    string = ""
    for i in range(0, len(query_relevant_terms[key])):
        string += " "+query_relevant_terms[key][i]
    modified_query = queries[key] +string
    line = str(key) + ". " + modified_query + "\n"
    outF.write(line)
outF.close()